In [1]:
persons = ["Michael Alan Weiner", "Billy Joel"]
prompt_template = "who is {person}?"
prompts = []
for person in persons:
    prompt = prompt_template.format(person=person)
    prompts.append(prompt)
    # generate response
    # generate claim set
    # generate candidate responses
    break

print(f"prompts:\n{prompts}")

original_response = "Michael Alan Weiner (born March 31, 1942) is an American radio host. Michael Alan Weiner is the host of The Savage Nation. Michael Alan Weiner was a good student when he was in America."
responses = []
responses.append(original_response)
print(f"responses:\n{responses}")

claim_set = ["Michael Alan Weiner (born March 31, 1942) is an American radio host.", "Michael Alan Weiner is the host of The Savage Nation.", "Michael Alan Weiner was a good student when he was in America."]

claim_sets = []
claim_sets.append(claim_set)
print(f"claim_sets:\n{claim_sets}")

# Other candidate responses generated by the same LLM to perform self-check for consistency
sample1 = "Michael Alan Weiner (born March 31, 1942) is an American radio host. He is the host of The Savage Country."
sample2 = "Michael Alan Weiner (born January 13, 1960) is a Canadian radio host. He works at The New York Times."
sample3 = "Michael Alan Weiner (born March 31, 1942) is an American radio host. He obtained his PhD from MIT."

candidate_responses = []
candidate_responses.append(sample1)
candidate_responses.append(sample2)
candidate_responses.append(sample3)

sampled_responses = []
sampled_responses.append(candidate_responses)
print(f"sampled_responses:\n{sampled_responses}")

prompts:
['who is Michael Alan Weiner?']
responses:
['Michael Alan Weiner (born March 31, 1942) is an American radio host. Michael Alan Weiner is the host of The Savage Nation. Michael Alan Weiner was a good student when he was in America.']
claim_sets:
[['Michael Alan Weiner (born March 31, 1942) is an American radio host.', 'Michael Alan Weiner is the host of The Savage Nation.', 'Michael Alan Weiner was a good student when he was in America.']]
sampled_responses:
[['Michael Alan Weiner (born March 31, 1942) is an American radio host. He is the host of The Savage Country.', 'Michael Alan Weiner (born January 13, 1960) is a Canadian radio host. He works at The New York Times.', 'Michael Alan Weiner (born March 31, 1942) is an American radio host. He obtained his PhD from MIT.']]


In [2]:
from uqlm.utils import NLI

nli = NLI()

In [19]:
probs = nli.predict(hypothesis="I am a boy", premise="I am a big boy")
print(probs)
entailment = nli.predict(hypothesis="I am a boy", premise="I am a big boy", return_probabilities=False)
print(entailment)

[[7.4262469e-04 9.8663443e-01 1.2622972e-02]]
False


In [20]:
probs.argmax()

1

In [21]:
probs.argmax(axis=1)[0]

1

## LUQ Scorer

In [2]:
from uqlm.black_box.luq import LUQScorer

In [3]:
luq_scorer = LUQScorer()

In [4]:
scores = luq_scorer.evaluate(claim_sets, sampled_responses)
scores

ClaimScores(response_scores=array([0.44444444]), claim_scores=[array([0.66666667, 0.16666667, 0.5       ])], entailment_scores=[array([[1. , 0. , 1. ],
       [0. , 0. , 0.5],
       [0.5, 0.5, 0.5]])])

# LongFormUQ

In [5]:
from uqlm.scorers import LongFormUQ
from langchain_google_vertexai import ChatVertexAI

In [6]:
test_llm = ChatVertexAI(model="gemini-2.5-flash", temperature=0)

lfuq = LongFormUQ(claim_decomposition_llm=test_llm, generation_llm=test_llm)

Score Only

In [7]:
# Score Only
# result  = lfuq.score(responses=responses, sampled_responses=sampled_responses)
# result.data

Generate and score

In [8]:
results_ = await lfuq.generate_and_score(prompts=prompts, num_responses=2)

Output()

In [9]:
# generate and score
results_.data

{'responses': ['Michael Alan Weiner is better known by his professional name, **Michael Savage**.\n\nHe is a prominent American radio host, author, and political commentator. He is known for his conservative, often controversial, and nationalist views. He hosts the nationally syndicated talk show, *The Savage Nation*.\n\nHe holds a Ph.D. in nutritional ethnomedicine from the University of California, Berkeley. Before his radio career, he was involved in environmentalism and health food. He is a highly influential figure in conservative media, known for his passionate and often provocative commentary.'],
 'claim_sets': [['Michael Alan Weiner is known by his professional name.',
   "Michael Alan Weiner's professional name is Michael Savage.",
   'Michael Alan Weiner is Michael Savage.',
   'Michael Savage is a prominent American radio host.',
   'Michael Savage is a prominent American author.',
   'Michael Savage is a prominent American political commentator.',
   'Michael Savage is know